In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import sys

# Print function that forces output to be flushed immediately
def log(message):
    print(message, flush=True)
    sys.stdout.flush()

# Data dictionary with updated company names
companies_data = {
    "Company 1": {"Open": "139.41", "High": "148.50", "Low": "138.10", "Close": "147.79", "Volume": "6914449"},
    "Company 2": {"Open": "512.49", "High": "523.50", "Low": "502.00", "Close": "507.77", "Volume": "838859"},
    "Company 3": {"Open": "281.00", "High": "286.00", "Low": "279.00", "Close": "285.29", "Volume": "863049"},
    # Add other companies as needed
}

log("Script starting...")

# Setup without using webdriver_manager
try:
    log("Setting up WebDriver directly...")
    options = webdriver.ChromeOptions()
    # Add options to make browser more stable
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-extensions")
    
    # Try to create the driver directly without webdriver_manager
    driver = webdriver.Chrome(options=options)
    log("WebDriver initialized successfully")
except Exception as e:
    log(f"ERROR setting up WebDriver: {e}")
    sys.exit(1)

try:
    # Step 1: Go to login page directly
    log("Navigating to login page...")
    driver.get("https://sampleloginpage.com/auth/login")  # Sample URL
    time.sleep(2)
    log("Login page loaded")

    # Step 2: Login
    log("Attempting login...")
    try:
        driver.find_element(By.NAME, "email").send_keys("sampleemail@example.com")  # Sample email
        driver.find_element(By.NAME, "password").send_keys("SamplePassword123")  # Sample password
        driver.find_element(By.XPATH, "//button[contains(text(),'Login')]").click()
        time.sleep(3)
        log("Login successful")
    except Exception as e:
        log(f"ERROR during login: {e}")
        raise

    # Step 3: Go to the stocks form
    log("Navigating to stocks page...")
    driver.get("https://samplestocksform.com/admin/stocks")  # Sample URL
    time.sleep(2)
    log("Stocks page loaded")

    # Count for tracking progress
    company_count = 0
    successful_submissions = 0
    
    # Loop through the companies data and submit the form for each company
    log(f"Starting to process {len(companies_data)} companies...")
    
    for company, data in companies_data.items():
        company_count += 1
        log(f"Processing company {company_count}/{len(companies_data)}: {company}")
        
        # Wait for the company dropdown to be available
        try:
            wait = WebDriverWait(driver, 10)
            company_dropdown = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "select")))

            log(f"Company dropdown found for {company}")
        except Exception as e:
            log(f"ERROR finding company dropdown for {company}: {e}")
            continue

        # Select the company from the dropdown
        try:
            Select(company_dropdown).select_by_visible_text(company)
            log(f"Selected company: {company}")
        except Exception as e:
            log(f"ERROR selecting company {company}: {e}")
            # Take screenshot to help debug
            try:
                driver.save_screenshot(f"error_selecting_{company_count}.png")
                log(f"Screenshot saved as error_selecting_{company_count}.png")
            except:
                log("Could not save screenshot")
            continue

        # Fill the stock data form
        try:
            driver.find_element(By.NAME, "open").send_keys(data["Open"])
            driver.find_element(By.NAME, "high").send_keys(data["High"])
            driver.find_element(By.NAME, "low").send_keys(data["Low"])
            driver.find_element(By.NAME, "close").send_keys(data["Close"])
            driver.find_element(By.NAME, "volume").send_keys(data["Volume"])
            driver.find_element(By.XPATH, "//button[contains(text(),'Submit')]").click()
            time.sleep(2)
            successful_submissions += 1
            log(f"Successfully submitted data for {company}")
        except Exception as e:
            log(f"ERROR submitting data for {company}: {e}")

    # Summary of submissions
    log(f"Successfully submitted {successful_submissions}/{len(companies_data)} companies.")

except Exception as e:
    log(f"ERROR during script execution: {e}")

finally:
    # Close the driver
    driver.quit()
